### 필요한 모듈 불러오기

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

import time
from tqdm.notebook import tqdm

import datetime
from datetime import timedelta

### 데이터 불러오기

In [105]:
data =pd.read_json('data_v1_COM.json',orient='table')

In [107]:
for i in range(0,len(data)):
    if data["브랜드"][i]=="Nobrand":
        data["브랜드"][i]=np.nan

In [108]:
data.dropna(inplace=True) ## 브랜드가 없는 행은 지우고 병합 --> 추후 다시 병합예정

### 브랜드 검색률 추가

In [109]:
import copy

In [110]:
a = copy.deepcopy(data[data.columns.difference(["상품종류","브랜드상품"])])
len(a)

36113

In [111]:
data.columns

Index(['방송일시', '노출(분)', '마더코드', '상품코드', '상품명', '상품군', '판매단가', '취급액', '브랜드',
       '상품종류', '브랜드상품', '수량', '방송날짜', '방송_월', '방송_일', '판매일수'],
      dtype='object')

In [112]:
a["방송날짜"] = pd.to_datetime(a['방송날짜']).dt.tz_localize(None)

In [113]:
import glob
final = pd.DataFrame(columns=a.columns) ## 최종데이터
상품군 = pd.unique(a["상품군"])
a["방송날짜"] = pd.to_datetime(a["방송날짜"])

for i in tqdm(range(0,len(상품군))):
    ## 파일 실행 위치에 맞게 변경
    path = r'C:/Users/SEC/Desktop/매진임박/브랜드겁색_TRAIN/'+상품군[i]
    all_files = glob.glob(path + "/*.xlsx")
    noth = pd.DataFrame(columns=["방송날짜","검색률","브랜드"])
    for filename in all_files:
        df = pd.read_excel(filename, header=6)
        new = pd.DataFrame({"방송날짜" : df["날짜"][-1:], "검색률" : df[df.columns[1]][-8:-1].mean(),"브랜드":df.columns[1]})
        noth = pd.concat([noth,new])    

    
    mg1 = a[a["상품군"]==상품군[i]][a["브랜드"]==noth["브랜드"].unique().tolist()[0]]
    mg1["방송날짜"] = pd.to_datetime(mg1["방송날짜"])
    mg2 = noth[noth["브랜드"]==noth["브랜드"].unique().tolist()[0]].reset_index(drop=True)
    mg2["방송날짜"] = pd.to_datetime(mg2["방송날짜"])
    new = pd.merge(mg1,mg2,how='left').fillna(method='ffill')
    merge = pd.DataFrame(columns=new.columns)
    merge = pd.concat([merge,new])
    

    for j in noth["브랜드"].unique().tolist()[1:]:
        mg1 = a[a["상품군"]==상품군[i]][a["브랜드"]==j]
        mg1["방송날짜"] = pd.to_datetime(mg1["방송날짜"])
        mg2 = noth[noth["브랜드"]==j].reset_index(drop=True)
        mg2["방송날짜"] = pd.to_datetime(mg2["방송날짜"])
        new = pd.merge(mg1,mg2,how='left').fillna(method='ffill')
        merge = pd.concat([merge,new])
        
    final = pd.concat([final,merge])
    print(len(merge), 상품군[i])

4340 의류
3901 속옷
6529 주방
2029 농수축
1305 이미용
6479 가전
2863 생활용품
559 건강기능
3891 잡화
928 무형
2614 가구
660 침구



In [117]:
total = pd.merge(a,final,how='left').drop_duplicates().fillna(0)

### 브랜드 + 상품명 검색률

In [119]:
import glob
final = pd.DataFrame(columns=a.columns) ## 최종데이터
상품군 = pd.unique(a["상품군"])
a["방송날짜"] = pd.to_datetime(a["방송날짜"])

for i in tqdm(range(0,len(상품군))):
    ## 파일 실행 위치에 맞게 변경
    path = r'C:/Users/SEC/Desktop/매진임박/브랜드상품검색_TRAIN/'+상품군[i]
    all_files = glob.glob(path + "/*.xlsx")
    noth = pd.DataFrame(columns=["방송날짜","검색률2","브랜드"])
    for filename in all_files:
        df = pd.read_excel(filename, header=6)
        new = pd.DataFrame({"방송날짜" : df["날짜"][-1:], "검색률2" : df[df.columns[1]][-8:-1].mean(),"브랜드":df.columns[1]})
        noth = pd.concat([noth,new])
        
    noth = noth.reset_index(drop=True)
    for j in range(0,len(noth)):
        noth["브랜드"][j] = noth["브랜드"][j].replace(',', '')
        
    noth = noth.drop_duplicates()
    
    mg1 = a[a["상품군"]==상품군[i]][a["브랜드"]==noth["브랜드"].unique().tolist()[0]]
    mg1["방송날짜"] = pd.to_datetime(mg1["방송날짜"])
    mg2 = noth[noth["브랜드"]==noth["브랜드"].unique().tolist()[0]].reset_index(drop=True)
    mg2["방송날짜"] = pd.to_datetime(mg2["방송날짜"])
    new = pd.merge(mg1,mg2,how='left').fillna(method='ffill')
    merge = pd.DataFrame(columns=new.columns)
    merge = pd.concat([merge,new])
    

    for k in noth["브랜드"].unique().tolist()[1:]:
        mg1 = a[a["상품군"]==상품군[i]][a["브랜드"]==k]
        mg1["방송날짜"] = pd.to_datetime(mg1["방송날짜"])
        mg2 = noth[noth["브랜드"]==k].reset_index(drop=True)
        mg2["방송날짜"] = pd.to_datetime(mg2["방송날짜"])
        new = pd.merge(mg1,mg2,how='left').fillna(method='ffill')
        merge = pd.concat([merge,new])
        
    final = pd.concat([final,merge])
    print(len(merge), 상품군[i])

4340 의류
3901 속옷
6526 주방
2029 농수축
1305 이미용
6479 가전
2863 생활용품
559 건강기능
3891 잡화
928 무형
2614 가구
660 침구



In [123]:
total2 = pd.merge(total,final,how='left').drop_duplicates().fillna(0)

### 원본 데이터와 total 데이터 병합 후 브랜드이름 없는 행의 결측치 채우기

In [125]:
data2 =pd.read_json('data_v1_COM.json',orient='table')

data2["방송날짜"] = pd.to_datetime(data2["방송날짜"]).dt.tz_localize(None)

data_v4 = pd.merge(data2,total2,how='left')

- 브랜드 결측치   --> Nobrand
- 판매일수 결측치 --> 해당 상품군의 판매일수로 대체
- 검색률 결측치   --> fill_mean_func 함수를 통해 해당하는 상품군의 평균치로 대체

In [135]:
fill_mean_func = lambda g: g.fillna(g.mean())

data_v4 = data_v4.groupby('상품군').apply(fill_mean_func)
data_v4.reset_index(drop=True,inplace=True)

In [136]:
len(data_v4)
data_v4.isnull().sum()

38309

방송일시     0
노출(분)    0
마더코드     0
상품코드     0
상품명      0
상품군      0
판매단가     0
취급액      0
브랜드      0
상품종류     0
브랜드상품    0
수량       0
방송날짜     0
방송_월     0
방송_일     0
판매일수     0
검색률      0
검색률2     0
dtype: int64

### 최종 데이터

In [137]:
data_v4.to_json("data_v2_COM.json",orient ="table")